## InfoCepts Data Engineering Solution
#### By Arkadeepta Roy (@arcat_)

In [19]:
import pandas as pd
import numpy as np
from datetime import datetime

In [20]:
df_file_location="./infocept_xml_data_.xml"
df = pd.read_xml(df_file_location)  

In [21]:
df.head(5)

,index,user_session,total_clicks,total_items,total_cats,max_dwell,mean_dwell,total_duration,click_rate,day_of_week,...,cat_most_viewed,prod_most_viewed_n_times,prod_most_viewed,cat_views_freqs,prod_views_freqs,cat_buys_freqs,prod_buys_freqs,is_purchase,start_time_ts,end_time_ts
0,0,8b69208e-7c64-422d-b670-5b9d82d25606,7,2,1,0.511508,5.995878,166.711175,0.072524,1,...,2053013560982667520,1.0,1030297,0.267802,0.000945,2.337402e-01,0.000690,0,"October 07, 2019 Monday, 00:08:28","October 26, 2019 Saturday, 16:08:29"
1,1,8068596c-a2b8-46ca-8791-e10d70520a1f,7,2,1,7.626576,3.529165,0.434185,0.204248,0,...,2053013602328876544,1.0,36083228,0.140925,0.000946,4.550292e-01,0.000690,0,"October 01, 2019 Tuesday, 15:08:40","October 15, 2019 Tuesday, 20:03:18"
2,2,4d153217-e06d-4581-8924-702c7f757610,2,22,1,3.095637,180.977516,0.935320,0.045982,4,...,2175094838038844672,2.0,1002263,0.000294,0.000945,9.102017e-08,0.000690,0,"October 14, 2019 Monday, 21:17:24","October 23, 2019 Wednesday, 21:56:30"
3,3,e65a6869-fb45-4f34-8482-c80278676d0f,2,2,1,1.089128,4.338280,0.417055,0.028476,0,...,2053013568140556288,3.0,1003257,0.150107,0.000945,4.550279e-01,0.000739,0,"October 05, 2019 Saturday, 21:54:13","October 17, 2019 Thursday, 13:54:29"
4,4,d8206742-06d7-42a2-8163-d70f9dd6cb18,5,1,1,0.329814,3.180891,2.485720,0.094812,5,...,2053013558410492160,1.0,13728056,0.267784,0.000945,4.550284e-01,0.000690,0,"October 09, 2019 Wednesday, 13:33:22","October 29, 2019 Tuesday, 11:51:04"


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 24 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   index                     500000 non-null  int64  
 1   user_session              500000 non-null  object 
 2   total_clicks              500000 non-null  int64  
 3   total_items               500000 non-null  int64  
 4   total_cats                500000 non-null  int64  
 5   max_dwell                 500000 non-null  float64
 6   mean_dwell                500000 non-null  float64
 7   total_duration            500000 non-null  float64
 8   click_rate                500000 non-null  float64
 9   day_of_week               500000 non-null  int64  
 10  is_weekend                500000 non-null  int64  
 11  is_special_day            500000 non-null  int64  
 12  time_of_day               500000 non-null  int64  
 13  cat_most_viewed_n_times   500000 non-null  f

In [23]:
df.isnull().sum()

index                       0
user_session                0
total_clicks                0
total_items                 0
total_cats                  0
max_dwell                   0
mean_dwell                  0
total_duration              0
click_rate                  0
day_of_week                 0
is_weekend                  0
is_special_day              0
time_of_day                 0
cat_most_viewed_n_times     0
cat_most_viewed             0
prod_most_viewed_n_times    0
prod_most_viewed            0
cat_views_freqs             0
prod_views_freqs            0
cat_buys_freqs              0
prod_buys_freqs             0
is_purchase                 0
start_time_ts               0
end_time_ts                 0
dtype: int64

#### We can see that there are no null values in the dataset. We can proceed for further exploration

In [24]:
df.day_of_week.value_counts()

5    260104
0    126565
1     42915
3     24829
4     19270
2     17302
6      9015
Name: day_of_week, dtype: int64

#### Sunday and Friday have the most records.

## Dataframe for submission

In [25]:
sample_solution_file_location="./infocept_submission.csv"
sol_df = pd.read_csv(sample_solution_file_location)

#### Pre processing : Adding columns which have no changes in the new df
##### 'is_special_day', 'prod_views_freqs' and 'prod_buys_freqs' columns.

In [26]:
# sol_df['is_special_day'] = df['is_special_day']
# sol_df['prod_views_freqs'] = df['prod_views_freqs']
# sol_df['prod_buys_freqs'] = df['prod_buys_freqs']

#### 1. Process the "start_time_ts" column to create "time_spec_points" column. Allot exact points equaling the time of the day the user is surfing, but allot extra 5 points if they are surfing after 4 o clock in the afternoon. For example: If the user starts surfing at 9 o clock in the morning allot 9 points, but user surfing after 4 o clock in the afternoon will receive 21 points.

In [27]:
def calculate_time_points(row):  
    points = pd.to_datetime(row).hour
    if points >= 16:
        points += 5
    return points
 
sol_df["time_spec_points"] = df["start_time_ts"].apply(lambda val: calculate_time_points(val)) 

#### 2. Reduce the "total_duration" round to 2 decimal places: example (166.711175 -> 166.71) (0.935320 -> 0.94)

In [28]:
sol_df["total_duration"] = df["total_duration"].apply(lambda val: round(val, 2))

#### 3.  Add the values of columns  "total_clicks", "total_items", "total_cats" to create a new column: "total_inventory". If the value "total_inventory" is greater than 10, then make sure the value is increased by 100%.

In [29]:
def calculate_total_inventory(row):  
    total_inventory = row["total_clicks"] + row["total_items"] + row["total_cats"]
    if total_inventory > 10:
        total_inventory *=2
    return total_inventory
sol_df["total_inventory"] = df.apply(lambda row: calculate_total_inventory(row),axis=1) 

#### 4. Create a new column "give_big_discount". The value of '"give_big_discount" should be 1 if day_of_week is 0 and is_special_day is 1, otherwise 0.

In [30]:
sol_df["give_big_discount"] = df.apply(lambda row: 1 if row["day_of_week"] == 0 and row["is_special_day"] == 1 else 0, axis=1) 

#### 5. Create a new column prod_views_buys_ratio which should be a ratio of prod_views_freqs and prod_buys_freqs

In [31]:
sol_df["prod_views_buys_ratio"] = df.apply(lambda row: round(row["prod_views_freqs"] / row["prod_buys_freqs"],2), axis=1) 

#### 6. Create a new column create loyalty_points which should be calculated based on the following conditions: if a user is spending more than 3 seconds on a Sunday -- and loyalty points score will be 10 multiplied seconds over 3. 

In [32]:
def calculate_loyalty_points(row):  
    loyalty_points = 0
    if row["day_of_week"] == 0 and row["total_duration"] > 3:
        loyalty_points = 10*(row["total_duration"] - 3)
    return round(loyalty_points,2)
 
sol_df["loyalty_points"] = df.apply(lambda row: calculate_loyalty_points(row), axis=1) 

#### 7. Check the submission file for column details.

In [33]:
sol_df.head()

,total_duration,is_special_day,prod_views_freqs,prod_buys_freqs,give_big_discount,prod_views_buys_ratio,loyalty_points,total_inventory,time_spec_points
0,166.71,0,0,0,0,1.37,0.0,10,0
1,0.43,0,0,0,0,1.37,0.0,10,15
2,0.94,0,0,0,0,1.37,0.0,50,26
3,0.42,0,0,0,0,1.28,0.0,5,26
4,2.49,0,0,0,0,1.37,0.0,7,13


In [34]:
sol_df.tail()

,total_duration,is_special_day,prod_views_freqs,prod_buys_freqs,give_big_discount,prod_views_buys_ratio,loyalty_points,total_inventory,time_spec_points
499995,0.22,0,0,0,0,1.37,0.0,10,21
499996,6.56,0,0,0,0,1.37,0.0,5,6
499997,0.02,0,0,0,0,1.39,0.0,10,9
499998,0.02,0,0,0,0,1.37,0.0,7,23
499999,0.02,0,0,0,0,1.39,0.0,6,3


### Submission

In [35]:
now = datetime.now()
dt_string = now.strftime("_%d%m%Y_%H%M%S")

In [36]:
sol_df.to_csv('infocept_solution_'+dt_string+'.csv', index=False, header=True)  